<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/reddit_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# bringing in the client_id and the secret for the reddit
from google.colab import files
import pandas as pd



In [ ]:
files.upload()
# erased the output so that the client_id and the secret are not visible

In [4]:
with open("udemy_reddit_api.txt", mode="r") as f:
  while True:
    line = f.readline()
    if line == "":
      break
    # splitting the line
    splitLine  = line.split()
   
    if "personal_use_script:" in splitLine:
      client_id = splitLine[1]
    elif "secret:" in splitLine:
      secret = splitLine[1]
    elif "password:" in splitLine:
      reddit_pass = splitLine[1]
    elif "username:" in splitLine:
      reddit_user = splitLine[1] 



In [5]:
# requesting an oauth using request
import requests

In [ ]:
# doing the authenitaction
auth = requests.auth.HTTPBasicAuth(client_id, secret)

In [ ]:
post_data = {"grant_type": "password", 
             "username": reddit_user, 
             "password": reddit_pass}

In [ ]:
headers = {"User-Agent": "myUdemyTutoria/0.0.1"}

In [ ]:
res = requests.post("https://www.reddit.com/api/v1/access_token",
                    data=post_data, auth=auth, headers=headers)
res

<Response [200]>

In [ ]:
token = res.json()["access_token"]
token

'1064711172543-J7JEgn4i8Wkvao74CDJmmf1TTHwcIQ'

In [ ]:
# add this to the headers
headers["Authorization"] = f"bearer {token}"
headers

In [ ]:
res = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)
res

<Response [200]>

In [ ]:
res.status_code

200

In [ ]:
res.status_code == 200

True

In [21]:
# making a function that will do the same thing to get us
# contacted to the reddit api so that we can then make calls
def connect_reddit(reddit_username, reddit_password, app_or_client_id, secret, user_agent:str):
  """
  This function will return the headers dictionary 
  that now has the token in the headers so that it is available to use with
  any another get requests or post requests
  """
  # first will get the token to do the authorization
  # will make the auth for the requests
  auth = requests.auth.HTTPBasicAuth(client_id, secret)
  # making the header that tells reddidt what the User-Agent is
  headers = {"User-Agent": user_agent}
  # now making the post data dictionary
  post_data = {"grant_type": "password", 
               "username": reddit_username,
               "password": reddit_password}
  # Will then try to get the token
  res = requests.post("https://www.reddit.com/api/v1/access_token", 
                      data=post_data,
                      auth=auth,
                      headers=headers)
  if res.status_code != 200:
    raise Exception(f"Unable to get the token to make a connection to Reddit the responce is {res}")
  # now returning the headers which will contain the access_token
  token = res.json()["access_token"]
  headers["Authorization"] = f"bearer {token}"
  return headers



In [22]:
# running the function that will give the headers
headers = connect_reddit(reddit_username=reddit_user, reddit_password=reddit_pass,
                         app_or_client_id= client_id, secret=secret,
                         user_agent="myUdemyTutorial/0.0.1")

In [ ]:
# trying the headers on the api to just me
res = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)
res

<Response [200]>

In [ ]:
# setting the api -- the address where we will go to 
api = "https://oauth.reddit.com"

In [ ]:
# getting some of the investing data from reddit
res = requests.get(f"{api}/r/investing/new", headers=headers, 
                   params={"limit": "100"})
res

<Response [200]>

In [8]:
# making the list of the keys that we want to use to get 
# the data out of the repsonce
keep_list = ["name", "created_utc", "subreddit", "title", "selftext", 
             "upvote_ratio", "ups", "downs", "score" ]

In [ ]:
# making a dataframe with the keep_keys as the columns
df = pd.DataFrame(columns=keep_list)
df.columns

Index(['name', 'created_utc', 'subreddit', 'title', 'selftext', 'upvote_ratio',
       'ups', 'downs', 'score'],
      dtype='object')

In [ ]:
res.json()["data"]["children"][0]["data"]

{'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': 'TareXmd',
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_5iplk',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': True,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'content_categories': None,
 'contest_mode': False,
 'created': 1627678700.0,
 'created_utc': 1627678700.0,
 'discussion_type': None,
 'distinguished': None,
 'domain': 'self.investing',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'gildings': {},
 'hidden': False,
 'hide_score': False,
 'id': 'outsbk',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True

In [10]:
# checking the speed of the running
import time

In [11]:
# will now loop through the data and then put into the dataframe
for i in range(len(res.json()["data"]["children"])):
  # for each of the children we will then add all the information
  # into the dataframe
  start = time.time()
  theDict = res.json()["data"]["children"][i]["data"]
  insertDict = {"name":theDict["name"], "created_utc":theDict["created_utc"], 
                "subreddit":theDict["subreddit"], "title":theDict["title"],
                "selftext":theDict["selftext"], "upvote_ratio":theDict['upvote_ratio'],
                "ups":theDict['ups'], "downs":theDict["downs"], "score":theDict["score"]}
  
  # now doing append on the dataframe
  df = df.append(insertDict, ignore_index=True)
  
end = time.time()

# checking to see how long this method takes
t = end - start
print(f"The amount of time that it took to run this is {t}")
# now looking at the dataframe to see what is in it
print(f"the shape of the dataframe is {df.shape}")
df.tail()

NameError: ignored

In [20]:
# making it to loop through the data so that we can then collect 
# data from the reddit api
# will make a function that will do this
def get_reddit_data(headers, cols_to_get ,responce=None, url_to_call=None, subreddit=None, 
                    number_of_loops=-1, limit=100, df=None , df_len=None):
  """ 
  This is the function that will get data from reddit and will put it to a dataframe.

  Arg:

  "headers":  The headers used to get the responce.  The headers contain the Token and the User-Agent.


  cols_to_get:  This is a list of the keys that are found in the data that are wanted to 
  be gathered from the reddit data that is returned.  These are should be the same names as the colums 
  found in the dataFrame also so that it knows where to put them in the dataFrame.

  responce:  if we have made the first call through api to gather the data then, the 
  function will run from this reponce to get the data.  If the responce is None then this funtion will 
  use the "url_to_call" and the "subreddit"  and the "headers" to get the first repsonce.

  "url_to_call":  this is the url to call the api of reddit.  

  "subreddit":  this is the subreddit that we want to get the data from.

  
  "number_of_loops":  This is the number of times that this function will try to get data from reddit.  
  This function will get the lesser of either the "number_of_loops" or when reddit stops allowing gathering data.
  -1 for number_of_loops means that this function will try to gather data until reddit refuses.

  "limit":  This is the number of children to gather each of the get requests

   df: This is the dataFrame that the data is added to. If it is none then a new dataframe is created

   df_len:  Can choose a rough estimate of the size of the dataframe using the calls from reddit.
   When using this one number_of_loops will be set internally as -1.

  """

  if df_len:
    number_of_loops = -1

  if df == None:
    df = pd.DataFrame(columns=cols_to_get)

  # the global variable for the function
  after = None
  theParams = {"limit":str(limit)}
  counter = 0 # this is to count the number of times that the function will be 
              # gathering data from reddit

  if responce == None:
    if url_to_call == None or subreddit == None:
      raise Exception("Url and or the subreddit must not be None if no responce is passed in")

    # getting the first reponce
    responce = requests.get(url=f"{url_to_call}/r/{subreddit}", params=theParams, headers=headers)
  if responce.status_code != 200:
    raise Exception(f"The repsonce was not a 200 for getting data from reddit but was a {responce.status_code}")
  # now gathering the data from reddit for the number of requests
  while True:
  #   'data': {'after': 't3_opohad',
  # 'before': None,
  # 'children': [{'data': {'all_awardings': [],
  #    'allow_live_comments': False,
  #    'approved_at_utc': None,
    # will be setting the before and the after variable first 
    theDict = responce.json()

    after = theDict["data"]["after"]
    # getting the data from the responce
    # looping through all the children and then will put them into the dataframe
    for i in range(len(theDict["data"]["children"])):
      # incrementing the counter 
      counter +=1
      childDict = theDict["data"]["children"][i]["data"]
      # making the dictionary to append to the datframe
      dict_to_append = {theItem:childDict[theItem] for theItem in cols_to_get}
      # appending to the dataframe
      df = df.append(other=dict_to_append, ignore_index=True)

      if df_len < df.shape[0]:
        number_of_loops = 1
        break
    
    # now we are going to make the next call to reddit for more info
    if number_of_loops != -1 and counter >= number_of_loops:
      print(f"Leaving the function with the counter having reached {counter} and the number of loops is set at {number_of_loops}")
      break
    
    theParams["after"] = after # putting in the after param to tell what to return from reddit
    responce = requests.get(f"{url_to_call}/r/{subreddit}", params=theParams, headers=headers)

    # checking to see what the responce is here
    if responce.status_code != 200 or len(responce.json()["data"]["children"]) == 0:
      num_children= len(responce.json()["data"]["children"])
      print(f"have left the fucntion because the responce was {responce.status_code} or the number of children was {num_children}")
      break  # this should mean that reddit says that we are done getting data
  
  # out of the while loop
  return df

In [23]:
# using the method above to get the data
df = get_reddit_data(headers=headers, cols_to_get=keep_list, url_to_call="https://oauth.reddit.com", subreddit="investing/new", number_of_loops=110, df_len=10000)
print(df.shape)

Leaving the function with the counter having reached 10001 and the number of loops is set at 1
(10001, 9)


In [24]:
df.tail()

,name,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score
9996,t3_o808c7,1.624668e+09,investing,Is it normal to not get excited about the stoc...,I've read a lot about boring stocks:\n\n[https...,0.73,42,0,42
9997,t3_o7zdfv,1.624665e+09,investing,Investing in pharmaceutical stocks prior to PDUFA,It seems like one could look at the upcoming a...,0.58,2,0,2
9998,t3_o7yn4z,1.624662e+09,investing,I don’t think I understand Market Cap.,"Okay, maybe I'm just dull, but I don't get how...",0.69,12,0,12
9999,t3_o7yli2,1.624662e+09,investing,I invest based on quantitative sentiment score...,Hey guys! Posted a little while ago about my p...,0.71,10,0,10
10000,t3_o7yjao,1.624662e+09,investing,S&amp;P Global Luxury index - where to find al...,Does anybody know where to find **all** consti...,0.56,1,0,1


In [25]:
# saving the dataframe
# going to replace all the pipes with empty string 
df = df.replace(to_replace={"|":""}, regex=True)

In [ ]:
df.shape

(14092, 9)

In [27]:
# making it a csv with pipes
df.to_csv("reddit.csv", sep="|", index=False)